In [1]:
#데이터 전처리
import pandas as pd
data=pd.read_csv('C:/Users/HCJ/Desktop/2021_Summer/Finance_data/bank.csv')
Y=data[['TARGET']]
X=data.drop(['Unnamed: 0','TARGET'],axis=1)
X.iloc[:,[42,43,44,63,64,65,66,67]] = X.iloc[:,[42,43,44,63,64,65,66,67]].astype('category')

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.3,random_state=1004)

from imblearn.over_sampling import SMOTEN
smoten = SMOTEN()
X_smoten, Y_smoten = smoten.fit_resample(X_train, Y_train)

In [2]:
#FM, 샘플링 X
import xlearn as xl
import numpy as np
# DMatrix transition, if use field ,use must pass field map(an array) of features
xdm_train = xl.DMatrix(X_train, Y_train)
xdm_test = xl.DMatrix(X_test, Y_test)
# Training task
ffm_model = xl.create_ffm()  # Use factorization machine
# we use the same API for train from file
# that is, you can also pass xl.DMatrix for this API now
ffm_model.setTrain(xdm_train)    # Training data
ffm_model.setValidate(xdm_test)  # Validation data
# param:
#  0. regression task
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: mae
param = {'task':'binary', 'lr':0.2,
         'lambda':0.002, 'metric':'acc'}
# Start to train
# The trained model will be stored in model.out
ffm_model.fit(param, './model_dm.out')
# Prediction task
# we use the same API for test from file
# that is, you can also pass xl.DMatrix for this API now
ffm_model.setTest(xdm_test)  # Test data
ffm_model.setSign() # Convert output to 0,1
# Start to predict
# The output result will be stored in output.txt
# if no result out path setted, we return res as numpy.ndarray
Y_pred = ffm_model.predict("./model_dm.out")

#성능 평가
from sklearn import metrics
ffm_accuracy=metrics.accuracy_score(Y_test,Y_pred)
ffm_score=metrics.precision_recall_fscore_support(Y_test,Y_pred)

print(ffm_accuracy)
print(pd.DataFrame(ffm_score))

0.9929449820138869
              0      1
0      0.992945    0.0
1      1.000000    0.0
2      0.996460    0.0
3  35608.000000  253.0


C:\Users\HCJ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
#FM, 샘플링 O
# DMatrix transition, if use field ,use must pass field map(an array) of features
xdm_train1 = xl.DMatrix(X_smoten, Y_smoten)
# Training task
ffm_model1 = xl.create_ffm()  # Use factorization machine
# we use the same API for train from file
# that is, you can also pass xl.DMatrix for this API now
ffm_model1.setTrain(xdm_train1)    # Training data
ffm_model1.setValidate(xdm_test)  # Validation data
# param:
#  0. regression task
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: mae
param = {'task':'binary', 'lr':0.03,
         'lambda':0.0001, 'metric':'acc'}
# Start to train
# The trained model will be stored in model.out
ffm_model1.fit(param, './modeldm.out')
# Prediction task
# we use the same API for test from file
# that is, you can also pass xl.DMatrix for this API now
ffm_model1.setTest(xdm_test)  # Test data
ffm_model1.setSign() # Convert output to 0,1
# Start to predict
# The output result will be stored in output.txt
# if no result out path setted, we return res as numpy.ndarray
Y_pred1 = ffm_model.predict("./modeldm.out")

#성능 평가
from sklearn import metrics
ffm_smote_accuracy=metrics.accuracy_score(Y_test,Y_pred1)
ffm_smote_score=metrics.precision_recall_fscore_support(Y_test,Y_pred1)

print(ffm_smote_accuracy)
print(pd.DataFrame(ffm_smote_score))

0.8821282172833997
              0           1
0      0.997464    0.040046
1      0.883537    0.683794
2      0.937050    0.075661
3  35608.000000  253.000000
